In [2]:
!pip install -q sentence-transformers transformers torch

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 96.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 84.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 43.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 14.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 8.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 60.3 MB/s eta 0:00:00


In [3]:
!pip install onnx


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.6/17.6 MB 50.5 MB/s eta 0:00:00


In [4]:
import torch
from transformers import AutoTokenizer, AutoModel
import os
from pathlib import Path

def convert_sentence_transformer_to_onnx():
    """Convert all-MiniLM-L6-v2 to ONNX format"""

    model_name = "sentence-transformers/all-MiniLM-L6-v2"

    print(f"Loading model: {model_name}")
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    model = AutoModel.from_pretrained(model_name)
    model.eval()

    # Create dummy input for export
    dummy_text = "This is a sample sentence for ONNX conversion"
    dummy_input = tokenizer(
        dummy_text,
        return_tensors="pt",
        padding=True,
        truncation=True,
        max_length=256
    )

    # Create output directory
    output_dir = Path("./onnx_model")
    output_dir.mkdir(exist_ok=True)

    # Export to ONNX
    output_path = output_dir / "model.onnx"

    print("Converting to ONNX...")
    torch.onnx.export(
        model,
        (dummy_input['input_ids'], dummy_input['attention_mask']),
        str(output_path),
        input_names=['input_ids', 'attention_mask'],
        output_names=['last_hidden_state'],
        dynamic_axes={
            'input_ids': {0: 'batch_size', 1: 'sequence'},
            'attention_mask': {0: 'batch_size', 1: 'sequence'},
            'last_hidden_state': {0: 'batch_size', 1: 'sequence'}
        },
        opset_version=14,
        verbose=False
    )

    # Save tokenizer files
    tokenizer.save_pretrained(str(output_dir))

    print(f"✅ Model converted successfully!")
    print(f"📁 Files saved to: {output_dir}/")

    # List created files
    for file in output_dir.iterdir():
        if file.is_file():
            print(f"   - {file.name}")

if __name__ == "__main__":
    convert_sentence_transformer_to_onnx()

# Zip the model folder for download
import shutil
shutil.make_archive("onnx_model", 'zip', "onnx_model")

from google.colab import files
files.download("onnx_model.zip")

Loading model: sentence-transformers/all-MiniLM-L6-v2
Converting to ONNX...
✅ Model converted successfully!
📁 Files saved to: onnx_model/
   - model.onnx
   - special_tokens_map.json
   - tokenizer.json
   - tokenizer_config.json
   - vocab.txt


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>